# Using cifar10 dataset

60000 tiny images of 10 classes. 
Uses up LOTTTTTT of CPU, therefore **TODO**: launch the same on GPU-enabled server at AWS

In [ ]:
# STEP 1: Import Data
import keras
from keras.datasets import cifar10
(X_train, y_train),(X_test, y_test) = cifar10.load_data()

print(X_train.shape)
print(X_test.shape)

The above SHAPE tells us:
    - Images are 32 x 32
    - Color images (bc of 3 = RGB = depth)

In [ ]:
#STEP 2: Visualize some of the data
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(20,5))
for i in range(36): # total of 36 images
    ax = fig.add_subplot(3,12, i+1, xticks=[], yticks=[]) # 3 rows, 12 columns
    ax.imshow(np.squeeze(X_train[i]))

In [ ]:
# STEP 3: Preprocessing (Rescale to [0,1] instead of [0,255])

X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [ ]:
#STEP 4: Break data into test, train, validation sets
from keras.utils import np_utils

#One-hot encoding LABELS
numOfLabels = len(np.unique(y_train))
y_train = np_utils.to_categorical(y_train, numOfLabels)
y_test = np_utils.to_categorical(y_test, numOfLabels)

#Train and validation split
(X_train, X_validation) = X_train[5000:], X_train[:5000]
(y_train, y_validation) = y_train[5000:], y_train[:5000]

#Shape of all 3 (train, test, validation)
print("Train size = ", X_train.shape[0])
print("Test size = ", X_test.shape[0])
print("Validate size = ", X_validation.shape[0])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

model = Sequential()
model.add(Flatten(input_shape = X_train.shape[1:]))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(numOfLabels, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint

chkpoint = ModelCheckpoint(filepath="MLP.weights.best.hdf5", verbose=1, save_best_only=True)
hist = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_validation, y_validation), callbacks=[chkpoint], verbose=0, shuffle=True)

In [ ]:
score = model.evaluation(X_test, y_test, verbose=0)
accuracy = score[1] *100

print("Test accuracy = ", accuracy, "%")

MLP is NOT a good idea for multiple reasons:
- Parameters are OVER 3 million (ie too much memory consumption)
- Test accuracy is faaaaaaaaaaaaaar low!

**Therefore, let's give CNN a try ---**

In [ ]:
from keras.layers import Conv2D, MaxPooling2D

cnn = Sequential()
cnn.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,3)))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Dropout(0.3))
cnn.add(Flatten())
cnn.add(Dense(500, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(10, activation='softmax'))

cnn.summary()

In [ ]:
cnn.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
chkpt = ModelCheckpoint(filepath="CNN.weights.best.hdf5",verbose=1,save_only_best=True)
hist = cnn.fit(X_train, y_train, callbacks=[chkpt], validation_data=(X_validation, y_validation), batch_size=32, epochs=100, shuffle=True, verbose=0)

In [ ]:
cnn.load_weights("CNN.weights.best.hdf5")

In [ ]:
score = cnn.evaluation(X_test, y_test, verbose=0)
accuracy = score[1]* 100

print ("Test accuracy (using CNN) = ", accuracy)